In [ ]:
%%capture
!pip install openai==0.28 -q
!pip install gradio==3.36.1
!pip install PyPDF2==2.12.1
!pip install plotly
!pip install plotly-express

In [ ]:
%%writefile style.css
 {
  max-width: 600px;
  margin-left: auto;
  margin-right: auto;
}

 {
  display: flex;
  align-items: center;
  justify-content: center;
}
gradio-app{
  background: rgb(153,0,255);
  background-image: radial-gradient(circle, rgba(153,0,255,1) 0%, rgba(9,15,121,1) 96%, rgba(2,0,36,1) 100%) !important;
  height: 100%;
  width: 100%;
}
.leftimage .rightimage{
  float:left;
  filter: drop-shadow(20px 20px 10px white);
}
.leftimage{
  padding-top:40px;
  margin-left:310px;
}
.rightimage{
  padding-top:40px;
  margin-right:320px;
}
a,
a:hover,
a:visited {
  text-decoration-line: underline;
  font-weight: 600;
  color: 1f2937 !important;
}

.dark a,
.dark a:hover,
.dark a:visited {
  color: f3f4f6 !important;
}

Writing style.css


In [ ]:
import gradio as gr
import PyPDF2
import os
import re
import plotly.graph_objects as go
import openai

class ResumeAnalyser:
    def __init__(self, api_key):
        openai.api_key = api_key

    def extract_text_from_file(self, file_path):
        file_extension = os.path.splitext(file_path)[1].lower()

        if file_extension == '.pdf':
            with open(file_path, 'rb') as file:
                reader = PyPDF2.PdfFileReader(file)
                extracted_text = ""
                for page_number in range(reader.getNumPages()):
                    page = reader.getPage(page_number)
                    extracted_text += page.extractText()
            return extracted_text
        elif file_extension == '.txt':
            with open(file_path, 'r') as file:
                return file.read()
        else:
            return "Unsupported file type"

    def response_from_ai(self, job_description_path, resume_path):
        job_description = self.extract_text_from_file(job_description_path)
        resume = self.extract_text_from_file(resume_path)

        conversation = [
            {"role": "system", "content": "You are a Resume Analyser"},
            {"role": "user", "content": f"""
            Given the job description and the resume, access the matching percentage to 100 and if 100 percentage not matched mention the remaining percentage with reason.if doesn't match return 0. **Job Description:**{job_description}**Resume:**{resume}
            **Detailed Analysis:**
            the result should be in this format:
            Matched Percentage: [matching percentage].
            Reason            : [give me a Reason (shortly in passage) and keys from job_description and resume get this matched percentage.].
            Skills To Improve : [give me a the skills How to improve and get 100 percentage job description matching (shortly in passage).].
            Keywords          : [give me a matched key words from {job_description} and {resume}].
            """}
        ]

        try:
            response = openai.ChatCompletion.create(
                model="gpt-3.5-turbo-16k",
                messages=conversation,
                temperature=0,
                max_tokens=500,
                n=1,
                stop=None,
            )
            generated_text = response['choices'][0]['message']['content']
            print(generated_text)
            return generated_text
        except Exception as e:
            print(f"Error in OpenAI request: {e}")
            return None

    def matching_percentage(self, job_description_path, resume_path):
        generated_text = self.response_from_ai(job_description_path.name, resume_path.name)
        print("generate response",generated_text)
        if generated_text:
            lines = generated_text.split('\n')

            matched_percentage = 0
            reason = ""
            skills_to_improve = ""
            keywords = ""

            for line in lines:
                if line.startswith('Matched Percentage:'):
                    matched_percentage = int(re.search(r'\d+', line).group())
                elif line.startswith('Reason'):
                    reason = line.split(':')[1].strip()
                elif line.startswith('Skills To Improve'):
                    skills_to_improve = line.split(':')[1].strip()
                elif line.startswith('Keywords'):
                    keywords = line.split(':')[1].strip()

            # Create a pie chart with plotly
            fig = go.Figure(data=[go.Pie(labels=['Matched', 'Remaining'], values=[matched_percentage, 100 - matched_percentage])])
            return matched_percentage, reason, skills_to_improve, keywords, fig
        else:
            return None, None, None, None, None

    def gradio_interface(self):
        with gr.Blocks(css="/content/style.css", theme="freddyaboulton/test-blue") as app:
            gr.HTML("""<center class="darkblue" text-align:center;padding:30px;'><center>
                <center><h1 class ="center" style="color:#fff">Department of Computer Science</h1></center>
                <br><center><h1 style="color:#fff">Resume Analyser</h1></center>""")

            with gr.Row():
                with gr.Column(scale=0.45, min_width=150):
                    file_jd = gr.File()
                    jobDescription = gr.UploadButton("Browse File", file_types=[".txt", ".pdf", ".doc", ".docx", ".json", ".csv"], elem_classes="filenameshow")
                with gr.Column(scale=0.45, min_width=150):
                    file_resume = gr.File()
                    resume = gr.UploadButton("Browse File", file_types=[".txt", ".pdf", ".doc", ".docx", ".json", ".csv"], elem_classes="filenameshow")
                with gr.Column(scale=0.10, min_width=150):
                    analyse = gr.Button("Analyse")
            with gr.Row():
                with gr.Column(scale=1.0, min_width=150):
                    perncentage = gr.Textbox(label="Matching Percentage", lines=8)
                with gr.Column(scale=1.0, min_width=150):
                    reason = gr.Textbox(label="Matching Reason", lines=8)
                with gr.Column(scale=1.0, min_width=150):
                    skills = gr.Textbox(label="Skills To Improve", lines=8)
                with gr.Column(scale=1.0, min_width=150):
                    keywords = gr.Textbox(label="Matched Keywords", lines=8)
            with gr.Row():
                with gr.Column(scale=1.0, min_width=150):
                    pychart = gr.Plot(label="Matching Percentage Chart")
            jobDescription.upload(self.filename, jobDescription, file_jd)
            resume.upload(self.filename, resume, file_resume)
            analyse.click(self.matching_percentage, [jobDescription, resume], [perncentage, reason, skills, keywords, pychart])

        app.launch(debug=True)

    def filename(self, uploadbutton):
        return uploadbutton.name

api_key = 'API-KEy'
resume = ResumeAnalyser(api_key)
resume.gradio_interface()

/usr/local/lib/python3.10/dist-packages/gradio_client/documentation.py:103: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/usr/local/lib/python3.10/dist-packages/gradio_client/documentation.py:103: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is reco

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Error in OpenAI request: Your account is not active, please check your billing details on our website.
generate response None
Error in OpenAI request: Your account is not active, please check your billing details on our website.
generate response None
